# Analyze Financial Data with Python Capstone Project

In [484]:
from math import sqrt, log
import numpy as np
import pandas as pd

In [485]:
def display_as_percentage(val):
    return '{:.1f}%'.format(val * 100)

## Import Financial Data

Importing CSV file with monthly adjusted closing prices from 2015 to 2020 for Disney (DIS), Tesla (TSLA), Materials Select Sector SPDR Fund (XLB), Bank of America (BAC), Redfin (RDFN), and Pfizer (PFE).

Please note: Redfin became public towards the end of July 2017 and thus no price info for the periods prior.

In [486]:
stock_df = pd.read_csv('stock_download.csv')
stock_df.head()

,Date,DIS,TSLA,XLB,BAC,RDFN,PFE
0,2015-01-31,87.463401,40.651100,42.438717,14.453347,NaN,24.560344
1,2015-02-28,95.434739,42.134631,44.944892,14.535460,NaN,25.976905
2,2015-03-31,98.779440,38.943636,44.155636,14.219095,NaN,26.377147
3,2015-04-30,100.235825,42.322287,44.274398,14.063897,NaN,26.739389
4,2015-05-31,102.575530,48.444100,45.310296,14.821879,NaN,26.354135


In [487]:
stock_df.tail()

,Date,DIS,TSLA,XLB,BAC,RDFN,PFE
67,2020-08-31,129.046665,360.059711,61.760113,25.488668,44.528334,35.958023
68,2020-09-30,129.428096,413.495239,63.826960,24.743030,48.260953,34.175059
69,2020-10-31,124.371819,427.209545,64.547040,24.221761,49.542727,34.595055
70,2020-11-30,138.880000,465.612498,68.889296,26.647415,44.306500,36.033518
71,2020-12-31,166.927272,638.818640,71.152610,29.207377,65.641818,38.948636


## Financial Statistics

Defining financial statistics functions.

In [488]:
def simple_returns(start_price, end_price, dividends = 0):
    simple_return = (end_price - start_price + dividends) / start_price
    return simple_return

def calculate_log_returns(start_price, end_price):
    log_r = log(end_price) - log(start_price)
    return log_r

def annualize_return(log_return, t):
    annual_r = log_return * t
    return annual_r

def calculate_variance(dataset):
    mean = sum(dataset)/len(dataset)
    numerator = 0
    
    for data in dataset:
        numerator += (data - mean) ** 2

    return numerator / len(dataset)

def calculate_stddev(dataset):
    variance = calculate_variance(dataset)
    return sqrt(variance)

### Simple Rate of Return

After inspecting the dataframe, I want to calculate the simple returns of the monthly prices and store as a dataframe with the returns shifted 1 row down.

In [489]:
simple_returns_df = stock_df #create duplicate dataframe to run function with
col = 1 #to omit the date column in the dataframe
total_columns = len(simple_returns_df.columns) #number of columns to iterate through
row_len = len(simple_returns_df) #number of closing price to iterate through

while col < total_columns:
    simple_r = []
    i = 1
    for row_value in simple_returns_df.iloc[:,col]:
        simple_r.append(simple_returns(row_value, simple_returns_df.iloc[i,col]))
        if i < row_len-1:
            i += 1
        else:
            break
    simple_returns_df[simple_returns_df.columns[col]] = pd.Series(simple_r)
    col += 1
    
simple_returns_df = simple_returns_df.shift()
simple_returns_df = simple_returns_df.drop(columns = 'Date')
simple_returns_df = simple_returns_df.drop(simple_returns_df.head(1).index)
simple_returns_df = simple_returns_df.reset_index(drop = True)

dates = stock_df
dates = dates['Date'].reset_index(drop = True)
dates = dates.drop(dates.head(1).index).reset_index(drop = True)
dates = dates.reset_index()

simple_returns_df = dates.join(simple_returns_df)
simple_returns_df = simple_returns_df.drop(columns = 'index')
simple_returns_df

,Date,DIS,TSLA,XLB,BAC,RDFN,PFE
0,2015-02-28,0.091139,0.036494,0.059054,0.005681,NaN,0.057677
1,2015-03-31,0.035047,-0.075733,-0.017561,-0.021765,NaN,0.015408
2,2015-04-30,0.014744,0.086757,0.002690,-0.010915,NaN,0.013733
3,2015-05-31,0.023342,0.144648,0.023397,0.053896,NaN,-0.014408
4,2015-06-30,0.015485,0.061024,-0.017341,0.047853,NaN,0.003652
...,...,...,...,...,...,...,...
66,2020-08-31,0.103911,0.232962,0.043095,0.076336,0.110382,0.066647
67,2020-09-30,0.002956,0.148407,0.033466,-0.029254,0.083826,-0.049585
68,2020-10-31,-0.039066,0.033167,0.011282,-0.021067,0.026559,0.012290
69,2020-11-30,0.116652,0.089893,0.067273,0.100144,-0.105691,0.041580


In [527]:
avg_returns = simple_returns_df.mean()
annual_returns = annualize_return(avg_returns, 12)
annual_returns = annual_returns.reset_index()
annual_returns = annual_returns.rename(columns = {'index':'Ticker', 0:'Annualized Return'})
annual_returns.sort_values('Annualized Return', ascending = False)

,Ticker,Annualized Return
1,TSLA,0.597161
4,RDFN,0.446835
3,BAC,0.153265
0,DIS,0.132603
2,XLB,0.101089
5,PFE,0.090895


Tesla and Redfin stocks have the highest annual returns in the 5 year analysis. Tesla has an annual return of 59.7% and Redfin has an annual return of 44.7%.

### Log Returns

Here I calculate the monthly log returns from 2015 to 2020. Log returns do not react to huge shift in adjusted closing prices as much and thus are lower versus simple rate of return. Again, I shifted the rows down 1 to have the date correlate with the end date price used.

In [491]:
stock_df = pd.read_csv('stock_download.csv')
log_return_df = stock_df
col = 1 #to omit the date column in the dataframe
total_columns = len(log_return_df.columns) #number of columns to iterate through
total_rows = len(log_return_df) #number of rows

while col < total_columns:
    log_r = []
    i = 1
    for row_value in log_return_df.iloc[:,col]:
        log_r.append(calculate_log_returns(row_value, log_return_df.iloc[i,col]))
        if i < total_rows - 1:
            i += 1
        else:
            break
    log_return_df[log_return_df.columns[col]] = pd.Series(log_r)
    col += 1

log_return_df = log_return_df.shift()
log_return_df = log_return_df.drop(columns = 'Date')
log_return_df = log_return_df.drop(log_return_df.head(1).index)
log_return_df = log_return_df.reset_index(drop = True)

log_return_df = dates.join(log_return_df)
log_return_df = log_return_df.drop(columns = 'index')
log_return_df

,Date,DIS,TSLA,XLB,BAC,RDFN,PFE
0,2015-02-28,0.087222,0.035844,0.057376,0.005665,NaN,0.056075
1,2015-03-31,0.034447,-0.078755,-0.017717,-0.022005,NaN,0.015290
2,2015-04-30,0.014636,0.083198,0.002686,-0.010975,NaN,0.013640
3,2015-05-31,0.023074,0.135097,0.023128,0.052493,NaN,-0.014513
4,2015-06-30,0.015367,0.059235,-0.017493,0.046743,NaN,0.003645
...,...,...,...,...,...,...,...
66,2020-08-31,0.098859,0.209420,0.042192,0.073563,0.104704,0.064520
67,2020-09-30,0.002951,0.138376,0.032918,-0.029690,0.080497,-0.050856
68,2020-10-31,-0.039850,0.032629,0.011219,-0.021292,0.026213,0.012215
69,2020-11-30,0.110335,0.086079,0.065107,0.095441,-0.111704,0.040739


In [492]:
avg_log_returns = log_return_df.mean()
annual_log_returns = annualize_return(avg_log_returns, 12)
annual_log_returns = annual_log_returns.reset_index()
annual_log_returns = annual_log_returns.rename(columns = {'index':'Ticker', 0:'Annualized Return'})
annual_log_returns

,Ticker,Annualized Return
0,DIS,0.109240
1,TSLA,0.465565
2,XLB,0.087341
3,BAC,0.118901
4,RDFN,0.308026
5,PFE,0.077934


Tesla and Redfin have the highest annualized log return at 46.6% and 30.8% respectively.

### Variance & Standard Dev Analysis

In [524]:
var_table = log_return_df.var().reset_index()
var_table = var_table.rename(columns = {'index':'Ticker', 0:'Variance'})
var_table.sort_values('Variance', ascending = False)

,Ticker,Variance
4,RDFN,0.022153
1,TSLA,0.019474
3,BAC,0.005902
0,DIS,0.003925
2,XLB,0.002325
5,PFE,0.002165


In [525]:
std_table = log_return_df.std().reset_index()
std_table = std_table.rename(columns = {'index':'Ticker', 0:'Std Dev'})
std_table.sort_values('Std Dev', ascending = False)

,Ticker,Std Dev
4,RDFN,0.148837
1,TSLA,0.139551
3,BAC,0.076825
0,DIS,0.062647
2,XLB,0.048213
5,PFE,0.046525


In [526]:
log_r_mean = log_return_df.mean().reset_index()
log_r_mean = log_r_mean.rename(columns = {'index':'Ticker', 0:'Mean'})
log_r_mean.sort_values('Mean', ascending = False)

,Ticker,Mean
1,TSLA,0.038797
4,RDFN,0.025669
3,BAC,0.009908
0,DIS,0.009103
2,XLB,0.007278
5,PFE,0.006495


Redfin and Tesla both have high returns; however, both are highly risky investments with a standard deviation of 14.9% and 14.0%, respectively.